In [1]:
import numpy as np
import torch
from torchinfo import summary

import modules.dataloaders as dataloaders
import modules.model_CNV_imagenet as cnv_model
import modules.model_CNV_AIMET_imagenet as cnv_aimet_model
import modules.metrics as metrics
import modules.val_final_model as val_final_model
import modules.utils as utils

In [2]:
# from brevitas.export import export_onnx_qcdq
from brevitas.export import export_qonnx

# Load Model

In [3]:
AIMET = "YES"
if AIMET == "NO":
    model = cnv_model.CNV().to('cuda')
elif AIMET == "YES":
    model = cnv_aimet_model.CNV_AIMET().to('cuda')
else:
    print("Wrong Model")

### Torchinfo

In [4]:
print(summary(model, input_size=(1, 3, 224, 224)))

/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/torch/_tensor.py:1362: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400410390/work/c10/core/TensorImpl.h:1900.)
  return super().rename(names)


Layer (type:depth-idx)                                                      Output Shape              Param #
CNV_AIMET                                                                   [1, 2]                    --
├─ModuleList: 1-87                                                          --                        (recursive)
│    └─QuantIdentity: 2-1                                                   [1, 3, 224, 224]          --
│    │    └─ActQuantProxyFromInjector: 3-1                                  [1, 3, 224, 224]          --
│    │    └─ActQuantProxyFromInjector: 3-2                                  [1, 3, 224, 224]          1
├─ModuleList: 1-88                                                          --                        (recursive)
│    └─QuantLinear: 2-138                                                   --                        (recursive)
│    │    └─WeightQuantProxyFromInjector: 3-172                             --                        (recursive)
├─ModuleList: 1

# Load Weights

In [5]:
# weights_folder = './experiments/test_v04__imagenetQuant__w2a4_PerChannel_FixedPoint/weights'
# model_ckpnt = weights_folder + '/BED_classifier__best_mean_F1.pt'

weights_folder = './experiments/test_v10__AIMET/weights'
model_ckpnt = weights_folder + '/BED_classifier__best_mean_F1.pt'

In [6]:
utils.load_checkpoint(
    model_path = model_ckpnt, 
    model = model,
    device = 'cuda'
)

Loading Model. Trained during 58 epochs


58

In [7]:
model.to('cpu')
model.eval();

# Test with DFire MINI Dataset: Train and Test

In [8]:
# import importlib
# importlib.reload(config)
# importlib.reload(dataloaders)

In [9]:
# train_dfire_mini_loader = dataloaders.get_dfire_mini_train_loader()
test_dfire_mini_loader = dataloaders.get_dfire_mini_test_loader()


TEST DFire MINI dataset
DFire Removed wrong images: 0
DFire empty images: 6
DFire only smoke images: 13
DFire only fire images: 2
DFire smoke and fire images: 9

Test dataset len: 30


### Test DFire MINI

In [10]:
with torch.no_grad():
    val_metrics = val_final_model.eval_fn(
        loader=test_dfire_mini_loader, 
        model=model,                         
        device='cpu')

Validating:   7%|█████████▍                                                                                                                                   | 2/30 [00:00<00:01, 17.68it/s]

Label tensor([[0., 0.]]) - Pred tensor([[-3.1803, -4.3637]])
Label tensor([[0., 0.]]) - Pred tensor([[-0.7817, -5.7509]])
Label tensor([[0., 1.]]) - Pred tensor([[-1.7971,  2.9322]])
Label tensor([[0., 1.]]) - Pred tensor([[-1.5452,  3.2201]])


Validating:  20%|████████████████████████████▏                                                                                                                | 6/30 [00:00<00:00, 27.90it/s]

Label tensor([[1., 0.]]) - Pred tensor([[ 2.8723, -7.1181]])
Label tensor([[1., 0.]]) - Pred tensor([[-0.8217, -6.6384]])
Label tensor([[1., 0.]]) - Pred tensor([[ 1.8528, -5.5110]])


Validating:  33%|██████████████████████████████████████████████▋                                                                                             | 10/30 [00:00<00:00, 30.95it/s]

...

Label tensor([[0., 0.]]) - Pred tensor([[-2.5047, -5.4950]])
Label tensor([[0., 0.]]) - Pred tensor([[-4.1038, -3.2763]])
Label tensor([[1., 1.]]) - Pred tensor([[6.0065, 4.7072]])
Label tensor([[1., 1.]]) - Pred tensor([[3.0042, 3.2481]])
Label tensor([[1., 1.]]) - Pred tensor([[2.8363, 4.5713]])
Label tensor([[1., 1.]]) - Pred tensor([[2.3805, 3.5959]])
Label tensor([[1., 1.]]) - Pred tensor([[7.0899, 4.8751]])


Validating:  87%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                  | 26/30 [00:00<00:00, 34.27it/s]

Label tensor([[1., 1.]]) - Pred tensor([[4.1436, 4.8831]])
Label tensor([[1., 1.]]) - Pred tensor([[6.8820, 6.7381]])
Label tensor([[1., 1.]]) - Pred tensor([[3.1561, 4.3355]])
Label tensor([[1., 0.]]) - Pred tensor([[ 2.2886, -6.5025]])
Label tensor([[1., 0.]]) - Pred tensor([[ 3.5879, -0.7537]])
Label tensor([[1., 0.]]) - Pred tensor([[ 3.4160, -3.1603]])
Label tensor([[1., 0.]]) - Pred tensor([[ 4.5553, -5.4790]])


Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 32.01it/s]

Label tensor([[1., 1.]]) - Pred tensor([[3.6318, 2.3326]])
Label tensor([[1., 0.]]) - Pred tensor([[-0.6298, -2.1249]])
SMOKE -> Precision: 1.0000 - Recall: 0.9091 - Accuracy: 0.9333 - F1: 0.9524
FIRE -> Precision: 1.0000 - Recall: 1.0000 - Accuracy: 1.0000 - F1: 1.0000


In [11]:
print('\nTesting with DFire MINI TEST after LOADING F1 Best Mean CHECKPOINT')  
print(val_metrics)


Testing with DFire MINI TEST after LOADING F1 Best Mean CHECKPOINT
{'Accuracy': [0.9333333373069763, 1.0], 'Precision': [1.0, 1.0], 'Recall': [0.9090909361839294, 1.0], 'F1': [0.9523809552192688, 1.0]}


# Convert the Model to BIPOLAR OUT

In [12]:
import brevitas.nn as qnn
import torch.nn as nn

In [13]:
class CNV_BIPOLAR_OUT(nn.Module):
    def __init__(self, base_model):
        super(CNV_BIPOLAR_OUT, self).__init__()
        self.base_model = base_model
        self.qnt_output = qnn.QuantIdentity(
            quant_type='binary', 
            scaling_impl_type='const',
            bit_width=1, min_val=-1.0, max_val=1.0)

    def forward(self, x):
        x = self.base_model(x)
        x = self.qnt_output(x)
        return x

In [14]:
cnv_bipolar_out = CNV_BIPOLAR_OUT(model).to('cpu')

### New Evaluation for BIPOLAR Out Model

In [15]:
from tqdm import tqdm

def eval_bipolar_fn(loader, model, device):
    
    model.eval()
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(device), y.to(device)
        yhat = model(x)

        # print(y.shape)
        # print(yhat.shape)
        
        #yhat[yhat < 1] = 0
        #yhat = torch.sigmoid(yhat.detach())
        # print(f'Label {y} - Pred {yhat}')
        metrics.precision_metric_cpu.update(yhat, y)
        metrics.recall_metric_cpu.update(yhat, y)
        metrics.accuracy_metric_cpu.update(yhat, y)
        metrics.f1_metric_cpu.update(yhat, y)
    
    precision = metrics.precision_metric_cpu.compute()
    recall = metrics.recall_metric_cpu.compute()
    accuracy = metrics.accuracy_metric_cpu.compute()
    f1 = metrics.f1_metric_cpu.compute()
    
    metrics.precision_metric_cpu.reset()
    metrics.recall_metric_cpu.reset()
    metrics.accuracy_metric_cpu.reset()
    metrics.f1_metric_cpu.reset()

    print(f'SMOKE -> Precision: {precision[0]:.4f} - Recall: {recall[0]:.4f} - Accuracy: {accuracy[0]:.4f} - F1: {f1[0]:.4f}')
    print(f'FIRE -> Precision: {precision[1]:.4f} - Recall: {recall[1]:.4f} - Accuracy: {accuracy[1]:.4f} - F1: {f1[1]:.4f}')
    
    return (
        {
        'Accuracy': [accuracy[0].item(), accuracy[1].item()],
        'Precision': [precision[0].item(), precision[1].item()],
        'Recall': [recall[0].item(), recall[1].item()],
        'F1': [f1[0].item(), f1[1].item()] 
        }
    )

### Mini Test

In [16]:
with torch.no_grad():
    val_metrics = eval_bipolar_fn(
        loader=test_dfire_mini_loader, 
        model=cnv_bipolar_out,                         
        device='cpu')

Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 30.97it/s]

SMOKE -> Precision: 1.0000 - Recall: 0.9091 - Accuracy: 0.9333 - F1: 0.9524
FIRE -> Precision: 1.0000 - Recall: 1.0000 - Accuracy: 1.0000 - F1: 1.0000


# Export Bipolar to QONNX

In [17]:
onnx_no_bipolar_filename = 'Best_F1_AIMET__NO_Bipolar.onnx'
export_qonnx(cnv_bipolar_out, torch.randn(1, 3, 224, 224), onnx_no_bipolar_filename);

In [18]:
onnx_bipolar_filename = 'Best_F1_AIMET__Bipolar.onnx'
export_qonnx(cnv_bipolar_out, torch.randn(1, 3, 224, 224), onnx_bipolar_filename);

# Evaluate QONNX Model

QONNX exec does not work. FINN exec must be used

In [19]:
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
import qonnx.core.onnx_exec as qonnx_exec
# from qonnx.transformation.infer_shapes import InferShapes
# from qonnx.transformation.fold_constants import FoldConstants
# from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

### Clean before evaluation

In [20]:
clean_no_bipolar_onnx_filename = 'Best_F1_AIMET__NO_Bipolar__CLEAN.onnx'
qonnx_cleanup(onnx_no_bipolar_filename, out_file=clean_no_bipolar_onnx_filename)

clean_bipolar_onnx_filename = 'Best_F1_AIMET__Bipolar__CLEAN.onnx'
qonnx_cleanup(onnx_bipolar_filename, out_file=clean_bipolar_onnx_filename)

In [21]:
def eval_qonnx(loader, model_filename):
    
    model_wrapped = ModelWrapper(model_filename)
    # model_wrapped = model_wrapped.transform(InferShapes())
    # model_wrapped = model_wrapped.transform(FoldConstants())
    # model_wrapped = model_wrapped.transform(GiveUniqueNodeNames())
    # model_wrapped = model_wrapped.transform(GiveReadableTensorNames())
    # model_wrapped = model_wrapped.transform(RemoveStaticGraphInputs())
    
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to('cpu'), y.to('cpu')
        img = x.detach().numpy()
        inp_dict = {"onnx::Mul_0": img}
        yhat = qonnx_exec.execute_onnx(model_wrapped, inp_dict)
        # print(y.shape)
        # print(yhat.shape)

        yhat = torch.tensor(yhat["global_out"], dtype=torch.float32)
        # print(f'Label {y} - Pred {yhat}')

        # yhat[yhat < 1] = 0
        yhat = torch.sigmoid(yhat.detach())

        print(f'Label {y} - Pred {yhat}')
        metrics.precision_metric_cpu.update(yhat, y)
        metrics.recall_metric_cpu.update(yhat, y)
        metrics.accuracy_metric_cpu.update(yhat, y)
        metrics.f1_metric_cpu.update(yhat, y)
    
    precision = metrics.precision_metric_cpu.compute()
    recall = metrics.recall_metric_cpu.compute()
    accuracy = metrics.accuracy_metric_cpu.compute()
    f1 = metrics.f1_metric_cpu.compute()
    
    metrics.precision_metric_cpu.reset()
    metrics.recall_metric_cpu.reset()
    metrics.accuracy_metric_cpu.reset()
    metrics.f1_metric_cpu.reset()

    print(f'SMOKE -> Precision: {precision[0]:.4f} - Recall: {recall[0]:.4f} - Accuracy: {accuracy[0]:.4f} - F1: {f1[0]:.4f}')
    print(f'FIRE -> Precision: {precision[1]:.4f} - Recall: {recall[1]:.4f} - Accuracy: {accuracy[1]:.4f} - F1: {f1[1]:.4f}')
    
    return (
        {
        'Accuracy': [accuracy[0].item(), accuracy[1].item()],
        'Precision': [precision[0].item(), precision[1].item()],
        'Recall': [recall[0].item(), recall[1].item()],
        'F1': [f1[0].item(), f1[1].item()] 
        }
    )

In [22]:
val_metrics = eval_qonnx(
    loader=test_dfire_mini_loader, 
    model_filename=clean_no_bipolar_onnx_filename)

Validating:   7%|█████████▍                                                                                                                                   | 2/30 [00:00<00:03,  8.24it/s]

Label tensor([[0., 0.]]) - Pred tensor([[0.2689, 0.2689]])
Label tensor([[0., 0.]]) - Pred tensor([[0.2689, 0.2689]])


Validating:  13%|██████████████████▊                                                                                                                          | 4/30 [00:00<00:02,  9.26it/s]

Label tensor([[0., 1.]]) - Pred tensor([[0.2689, 0.2689]])
Label tensor([[0., 1.]]) - Pred tensor([[0.2689, 0.2689]])
Label tensor([[1., 0.]]) - Pred tensor([[0.2689, 0.2689]])


Validating:  23%|████████████████████████████████▉                                                                                                            | 7/30 [00:00<00:02,  9.43it/s]

...

Label tensor([[1., 0.]]) - Pred tensor([[0.2689, 0.2689]])
Label tensor([[1., 0.]]) - Pred tensor([[0.2689, 0.2689]])


Validating:  93%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋         | 28/30 [00:03<00:00,  9.47it/s]

Label tensor([[1., 0.]]) - Pred tensor([[0.2689, 0.2689]])
Label tensor([[1., 0.]]) - Pred tensor([[0.2689, 0.2689]])


Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:03<00:00,  9.08it/s]

Label tensor([[1., 1.]]) - Pred tensor([[0.2689, 0.2689]])
Label tensor([[1., 0.]]) - Pred tensor([[0.2689, 0.2689]])
SMOKE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.2667 - F1: 0.0000
FIRE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.6333 - F1: 0.0000


In [23]:
val_metrics = eval_qonnx(
    loader=test_dfire_mini_loader, 
    model_filename=clean_bipolar_onnx_filename)

Validating:   7%|█████████▍                                                                                                                                   | 2/30 [00:00<00:03,  7.76it/s]

Label tensor([[0., 0.]]) - Pred tensor([[0.2689, 0.2689]])
Label tensor([[0., 0.]]) - Pred tensor([[0.2689, 0.2689]])


Validating:  13%|██████████████████▊                                                                                                                          | 4/30 [00:00<00:02,  8.98it/s]

Label tensor([[0., 1.]]) - Pred tensor([[0.2689, 0.2689]])
Label tensor([[0., 1.]]) - Pred tensor([[0.2689, 0.2689]])


Validating:  20%|████████████████████████████▏                                                                                                                | 6/30 [00:00<00:03,  7.48it/s]

...

Label tensor([[1., 0.]]) - Pred tensor([[0.2689, 0.2689]])
Label tensor([[1., 0.]]) - Pred tensor([[0.2689, 0.2689]])


Validating:  93%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋         | 28/30 [00:03<00:00,  7.63it/s]

Label tensor([[1., 0.]]) - Pred tensor([[0.2689, 0.2689]])
Label tensor([[1., 0.]]) - Pred tensor([[0.2689, 0.2689]])


Validating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:03<00:00,  8.67it/s]

Label tensor([[1., 1.]]) - Pred tensor([[0.2689, 0.2689]])
Label tensor([[1., 0.]]) - Pred tensor([[0.2689, 0.2689]])
SMOKE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.2667 - F1: 0.0000
FIRE -> Precision: 0.0000 - Recall: 0.0000 - Accuracy: 0.6333 - F1: 0.0000
